In [ ]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset


Saving kaggle.json to kaggle.json
100% 2.70G/2.70G [00:42<00:00, 38.2MB/s]
100% 2.70G/2.70G [00:42<00:00, 68.9MB/s]


In [ ]:
from zipfile import ZipFile
name='new-plant-diseases-dataset.zip'
with ZipFile(name,'r') as zip:
  zip.extractall()

In [ ]:
import tensorflow as tf
import keras 

In [ ]:
from keras.applications import ResNet50

In [ ]:
from keras.applications.resnet50 import preprocess_input

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
img_shape=[224,224]
resnet=ResNet50(input_shape=img_shape+[3],weights='imagenet',include_top= False )
for layer in resnet.layers:
  layer.trainable=False

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
from glob import glob
folder=glob('/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/*')

In [ ]:
x=keras.layers.Flatten()(resnet.output)
prediction=keras.layers.Dense(len(folder),activation="softmax")(x)
model=keras.models.Model(inputs=resnet.input,outputs=prediction)

In [ ]:
model.summary()
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

Model: "functional_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
train_datagen =tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 70295 images belonging to 38 classes.


In [ ]:
test_datagen =tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 17572 images belonging to 38 classes.


In [ ]:
history=model.fit_generator(training_set,epochs=10,steps_per_epoch=len(training_set),validation_data=test_set,validation_steps=len(test_set))

Epoch 1/10
2197/2197 [==============================] - 940s 428ms/step - loss: 3.3761 - accuracy: 0.4362 - val_loss: 3.3109 - val_accuracy: 0.4700
Epoch 2/10
2197/2197 [==============================] - 945s 430ms/step - loss: 3.1829 - accuracy: 0.4644 - val_loss: 4.4602 - val_accuracy: 0.4231
Epoch 3/10
2197/2197 [==============================] - 953s 434ms/step - loss: 3.1111 - accuracy: 0.4915 - val_loss: 2.4993 - val_accuracy: 0.5663
Epoch 4/10
2197/2197 [==============================] - 966s 440ms/step - loss: 3.0199 - accuracy: 0.5064 - val_loss: 3.0559 - val_accuracy: 0.5200
Epoch 5/10
2197/2197 [==============================] - 971s 442ms/step - loss: 3.0523 - accuracy: 0.5187 - val_loss: 2.8783 - val_accuracy: 0.5554
Epoch 6/10
2197/2197 [==============================] - 991s 451ms/step - loss: 3.0021 - accuracy: 0.5309 - val_loss: 3.6231 - val_accuracy: 0.4945
Epoch 7/10
2197/2197 [==============================] - 990s 451ms/step - loss: 2.8622 - accuracy: 0.5454 - val_

In [ ]:
model.save("model.h5")